In [1]:
from typing import Annotated, Literal, TypedDict
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage

from pinecone import Pinecone, ServerlessSpec

import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
PINECONE_INDEX_HOST = "https://funeral-index-wb03sz2.svc.aped-4627-b74a.pinecone.io"

In [ ]:
import traceback # 상세 에러 확인을 위해 추가
import os

@tool
def search_columbarium(query: str, region: str = None):
    """
    봉안당 또는 납골당을 검색할 때 사용합니다.
    
    Args:
        query (str): 검색할 질문 내용 (예: "시설 좋은 봉안당")
        region (str, optional): 사용자가 특정 지역을 언급한 경우 해당 지역명.
                                데이터베이스의 지역명은 '부산광역시', '충청북도' 등 광역자치단체 명칭입니다.
                                사용자가 '부산'이라고 하면 '부산광역시'로, '충북'이라고 하면 '충청북도'로 변환하여 입력하세요.
    """
    print(f"🔍 검색 시작 - 쿼리: {query}, 지역필터: {region}")
    
    # 1. 연결 설정
    embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
    pc = Pinecone(api_key=os.environ.get("PINECONE_API_KEY"))
    index = pc.Index(host=os.environ.get("PINECONE_INDEX_HOST"))
    vectorstore = PineconeVectorStore(index=index, embedding=embeddings)
    
    # 2. 메타데이터 필터 생성
    # Pinecone 필터 문법: {'meta_key': {'$eq': 'value'}}
    filter_dict = {}
    if region:
        filter_dict["region"] = {"$eq": region}
        print(f"✅ 지역 필터 적용됨: {region}")
    
    # 3. 필터를 적용하여 검색
    try:
        results = vectorstore.similarity_search(
            query, 
            k=3,
            filter=filter_dict  # 여기에 필터를 전달합니다
        )
    except Exception as e:
        return f"검색 중 오류가 발생했습니다: {str(e)}"

    # 4. 결과 반환
    if not results:
        return "해당 조건에 맞는 시설을 찾을 수 없습니다."

    formatted_results = ""
    for i, doc in enumerate(results):
        formatted_results += f"\n[결과 {i+1}]\n"
        formatted_results += f"이름: {doc.metadata.get('name')}\n"
        formatted_results += f"지역: {doc.metadata.get('region')} {doc.metadata.get('location')}\n"
        formatted_results += f"주소: {doc.metadata.get('address')}\n"
        formatted_results += f"전화번호: {doc.metadata.get('phone')}\n"
    
    return formatted_results

In [ ]:
# --- LangGraph 상태 및 노드 정의
tools = [search_columbarium]

# LLM 모델 로드 및 도구 바인딩
llm = ChatOpenAI(model="gpt-4o", temperature=0.1)
llm_with_tools = llm.bind_tools(tools)

# 그래프의 State 정의 (대화 기록 유지)
class AgentState(TypedDict):
    messages: Annotated[list, add_messages]

def agent_node(state: AgentState):
    # 현재까지의 대화 기록을 가져옴
    current_messages = state['messages']
    
    # 시스템 프롬프트 정의
    system_prompt = SystemMessage(content="""
    당신은 따뜻하고 사려 깊은 '장사 정보 안내 도우미'입니다.
    사용자가 장례식장, 봉안당, 납골당 등의 정보를 묻거나, 죽음/임종/빈소/조문 등과 관련된 상황을 암시하면 
    반드시 'search_columbarium' 도구를 사용하여 적절한 장소를 찾아주세요.
    
    사용자가 구체적인 지역을 말하지 않았다면, 먼저 어느 지역을 찾는지 정중하게 물어보세요.
    검색된 결과를 제공할 때는 위로의 말과 함께 정보를 정확하게 전달하세요.
    """)

    # 대화 기록 맨 앞에 시스템 메시지가 없으면, 이번 턴(invoke)에만 임시로 붙여서 보냄
    # (state에 영구 저장하지 않음으로써 중복 방지)
    if not isinstance(current_messages[0], SystemMessage):
        messages_to_send = [system_prompt] + current_messages
    else:
        messages_to_send = current_messages

    # LLM 호출
    response = llm_with_tools.invoke(messages_to_send)
    print("response:", response)
    
    # 새로운 응답만 리스트에 담아 반환 (add_messages가 알아서 기존 기록 뒤에 붙여줌)
    return {"messages": [response]}

# [Node 2] 도구 실행 노드
tool_node = ToolNode(tools)

# [Edge] 조건부 엣지: 도구를 쓸지, 답변을 끝낼지 결정
def should_continue(state: AgentState) -> Literal["tools", "__end__"]:
    messages = state['messages']
    last_message = messages[-1]

    print(f"Tool Calls 존재 여부: {last_message.tool_calls}")
    
    # LLM이 도구 호출을 요청했는지 확인
    if last_message.tool_calls:
        return "tools"
    return "__end__"

In [10]:
# --- 그래프 구성 (Workflow)
workflow = StateGraph(AgentState)

# 노드 추가
workflow.add_node("agent", agent_node)
workflow.add_node("tools", tool_node)

# 엣지 연결
workflow.set_entry_point("agent") # 시작점
workflow.add_conditional_edges(
    "agent",
    should_continue,
)
workflow.add_edge("tools", "agent") # 도구 실행 후 다시 에이전트로 복귀 (결과 해석)

# 그래프 컴파일
app = workflow.compile()

In [11]:
# --- 실행 테스트
def chat(user_input):
    print(f"\n사용자: {user_input}")
    inputs = {"messages": [HumanMessage(content=user_input)]}
    
    # 그래프 실행 (스트리밍으로 과정 확인 가능)
    for event in app.stream(inputs):
        for key, value in event.items():
            if key == "agent":
                msg = value["messages"][0]
                print(f"Agent 생각: {msg.content}") # 디버깅용
            elif key == "tools":
                # print(f"Tool 실행 결과...") # 디버깅용
                pass
    
    # 최종 응답 출력
    final_response = value["messages"][0].content
    print(f"챗봇: {final_response}")

In [1]:
# 케이스 1: 직접적인 요청
chat("서울에 있는 봉안당을 알려줘.")

NameError: name 'chat' is not defined

In [8]:


# # 케이스 2: 암시적인 상황 (도구 호출 전 지역 확인 유도 예상)
# chat("갑자기 아버지가 돌아가셨어.. 어떡해야 할지 모르겠네.")

# # 케이스 3: 암시적 상황 + 지역 포함
# chat("지금 부산 해운대 쪽인데, 급하게 빈소를 알아봐야 해.")

In [9]:
# chat("부산광역시에 있는 봉안당을 알려줘")